## 동적페이지 웹스크래핑
#### 사용자가 동작을 했을 때 동작하는 웹

In [18]:
import requests
from bs4 import BeautifulSoup

In [27]:
url = 'https://play.google.com/store/movies'
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
           "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
res = requests.get(url, headers=headers)
res.raise_for_status() #문제있으면 종료


soup = BeautifulSoup(res.text,'lxml')

### headers 정보를 내 컴퓨터로 안하면 다른 페이지 나올 수도 있음
### ex) 외국 사이트의 경우, 한국으로 접속안하면 영어로 뜸

#### 인기영화들 확인 

In [28]:
movie = soup.find_all('div',attrs={'class':"VfPpkd-WsjYwc VfPpkd-WsjYwc-OWXEXe-INsAgc KC1dQ Usd1Ac AaN0Dd  MPNOXb"})
print(len(movie))
#개수가 0개로 뜸

0


In [29]:
with open('movie.html', 'w',encoding='utf8') as f:#w는 읽기모드
    f.write(soup.prettify()) #html문서를 이쁘게 출력
    #f.wirte(res.text) 얘로하면 안이쁨 

### user-agent가 내꺼인데 내껄로 인식안되나? 그래서 안되는듯

In [30]:
movies = soup.find_all("div", attrs={"class":"ULeU3b neq64b"})
print(len(movies))

103


In [37]:
for movie in movies :
    title = movie.find(["div"], attrs = {"class" :"Epkrse"}).get_text()
    print(title)

범죄도시
공조2: 인터내셔날
인생은 아름다워
레드 아이
맞짱
인생은 뷰티풀: 비타돌체
투란도트: 어둠의 왕국
블로우백
몬스터 피라냐
돈텔파파
고속도로 가족
비스트
경성별희
깐부(코로나 방학소동)
죽어도 자이언츠
에브리띵 윌 체인지
로드 어헤드
아르키메데스의 대전
신 에반게리온 극장판
모자산책


AttributeError: 'NoneType' object has no attribute 'get_text'

### beautifulsoup으로 하면 상위에 있는 것들만 나오고 마우스 이동을 통해 나오는 것들은 못가져옴

## 셀레니움

In [44]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [55]:
browser = webdriver.Chrome('C:/Users/user/chromedriver.exe') #크롬드라이브 경로
browser.maximize_window()
url = 'https://play.google.com/store/movies'
browser.get(url)

<ipython-input-55-69b5409b44fe>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:/Users/user/chromedriver.exe') #크롬드라이브 경로


In [56]:
#지정한 위치로 스크롤 내리기
#모니터 높이인 1080으로 스크롤 내리기
#browser.execute_script('window.scrollTo(0,1080)') #1920x1080은 윈도우 크기. 

#화면 가장 아래로 스크롤 내리기
browser.execute_script('window.scrollTo(0,document.body.scrollHeight)') #현재 높이의 가장 아래로 내리기 

### 이것을 활용해 해당 페이지의 높이가 변하지 않을 때(마지막 페이지일 때까지) 스크롤을 가장 아래로 내리는 것을 반복

In [58]:
interval = 2 #2초에 한번식 스크롤 내림
#현재 문서 높이를 가져와서 저장
prev_h = browser.execute_script('return document.body.scrollHeight')

#반복 수행
while True:
    #스크롤 아래로 내림
    browser.execute_script('window.scrollTo(0,document.body.scrollHeight)') #현재 높이의 가장 아래로 내리기 
    time.sleep(interval)
    
    #현재 문서 높이를 가져와서 다시 저장
    curr_h = browser.execute_script('return document.body.scrollHeight')
    if curr_h == prev_h:
        break
    prev_h = curr_h
print('스크롤 완료')

스크롤 완료


## - 실제 셀레니움을 통해 동적페이지 스크래핑하기

In [60]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [68]:
browser = webdriver.Chrome('C:/Users/user/chromedriver.exe') #크롬드라이브 경로
browser.maximize_window()
url = 'https://play.google.com/store/movies'
browser.get(url)

<ipython-input-68-69b5409b44fe>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:/Users/user/chromedriver.exe') #크롬드라이브 경로


In [69]:
soup = BeautifulSoup(browser.page_source, "lxml")

movies = soup.find_all("a", attrs={"class":["Si6A0c ZD8Cqc","Si6A0c itIJzb"]}) #클래스가 list[]안에 있는 것과 전부 일치하는 것 가져옴
print(len(movies))

145


In [71]:
for movie in movies :
    title = movie.find(["div","span"], attrs = {"class" :"Epkrse"}).get_text() 
    
    #할인 전 가격
    ori_pr = movie.find("span", attrs = {"class":"SUZt4c P8AFK"})
    if ori_pr :
        ori_pr = ori_pr.get_text()
    else:
        print(title, "<할인되지 않는 영화 제외>")
        continue
    
    # 할인된 가격 
    pr = movie.find("span", attrs={"class":"VfPpfd VixbEe"}).get_text()
    
    #링크
    link = movie.find("a", attrs={"class":"Si6A0c ZD8Cqc"})["href"]
    # 올바른 링크 : https://play.google.com + link
    
    print(f"제목:{title}")
    print(f"할인 전 금액 :{ori_pr}")
    print(f"할인 후 금액 :{pr}")
    print("링크: ","https://play.google.com" + link)
    print("-"* 120)
    
browser.quit()

범죄도시 <할인되지 않는 영화 제외>
공조2: 인터내셔날 <할인되지 않는 영화 제외>
레드 아이 <할인되지 않는 영화 제외>
인생은 아름다워 <할인되지 않는 영화 제외>
맞짱 <할인되지 않는 영화 제외>
인생은 뷰티풀: 비타돌체 <할인되지 않는 영화 제외>
블로우백 <할인되지 않는 영화 제외>
투란도트: 어둠의 왕국 <할인되지 않는 영화 제외>
몬스터 피라냐 <할인되지 않는 영화 제외>
돈텔파파 <할인되지 않는 영화 제외>
고속도로 가족 <할인되지 않는 영화 제외>
비스트 <할인되지 않는 영화 제외>
경성별희 <할인되지 않는 영화 제외>
깐부(코로나 방학소동) <할인되지 않는 영화 제외>
죽어도 자이언츠 <할인되지 않는 영화 제외>
에브리띵 윌 체인지 <할인되지 않는 영화 제외>
로드 어헤드 <할인되지 않는 영화 제외>
아르키메데스의 대전 <할인되지 않는 영화 제외>


TypeError: 'NoneType' object is not subscriptable

In [74]:
for movie in movies :
    title = movie.find(["div","span"], attrs = {"class" :"DdYX5"}).get_text() 
    
    #할인 전 가격
    ori_pr = movie.find("span", attrs = {"class":"SUZt4c P8AFK"})
    if ori_pr :
        ori_pr = ori_pr.get_text()
    else:
        print(title, "<할인되지 않는 영화 제외>")
        continue
    
    #인기영화는 할인이 없어서 따로 원가 넣겠음(?)
    pop_pr = movie.find("span", attrs = {"class":"w2kbF ePXqnb"})
    if pop_pr : 
        print(title, "<할인되지 않는 영화 제외>")
    
    # 할인된 가격 
    pr = movie.find("span", attrs={"class":"VfPpfd VixbEe"}).get_text()
    
    #링크
    link = movie.find("a", attrs={"class":"Si6A0c itIJzb"})["href"]
    # 올바른 링크 : https://play.google.com + link
    
    print(f"제목:{title}")
    print(f"할인 전 금액 :{ori_pr}")
    print(f"할인 후 금액 :{pr}")
    #print("링크: ","https://play.google.com" + link)
    print("-"* 120)
    
browser.quit()

AttributeError: 'NoneType' object has no attribute 'get_text'

In [72]:
for movie in movies :
    title = movie.find(["div","span"], attrs = {"class" :["Epkrse","DdYX5"]}).get_text() 
    
    #할인 전 가격
    ori_pr = movie.find("span", attrs = {"class":"SUZt4c P8AFK"})
    if ori_pr :
        ori_pr = ori_pr.get_text()
    else:
        print(title, "<할인되지 않는 영화 제외>")
        continue
    
    #인기영화는 할인이 없어서 따로 원가 넣겠음(?)
    pop_pr = movie.find("span", attrs = {"class":"w2kbF ePXqnb"})
    if pop_pr : 
        print(title, "<할인되지 않는 영화 제외>")
    
    # 할인된 가격 
    pr = movie.find("span", attrs={"class":"VfPpfd VixbEe"}).get_text()
    
    #링크
    #link = movie.find("a", attrs={"class":["Si6A0c ZD8Cqc","Si6A0c itIJzb"]})["href"]
    # 올바른 링크 : https://play.google.com + link
    
    print(f"제목:{title}")
    print(f"할인 전 금액 :{ori_pr}")
    print(f"할인 후 금액 :{pr}")
    #print("링크: ","https://play.google.com" + link)
    print("-"* 120)
    
browser.quit()

범죄도시 <할인되지 않는 영화 제외>
공조2: 인터내셔날 <할인되지 않는 영화 제외>
레드 아이 <할인되지 않는 영화 제외>
인생은 아름다워 <할인되지 않는 영화 제외>
맞짱 <할인되지 않는 영화 제외>
인생은 뷰티풀: 비타돌체 <할인되지 않는 영화 제외>
블로우백 <할인되지 않는 영화 제외>
투란도트: 어둠의 왕국 <할인되지 않는 영화 제외>
몬스터 피라냐 <할인되지 않는 영화 제외>
돈텔파파 <할인되지 않는 영화 제외>
고속도로 가족 <할인되지 않는 영화 제외>
비스트 <할인되지 않는 영화 제외>
경성별희 <할인되지 않는 영화 제외>
깐부(코로나 방학소동) <할인되지 않는 영화 제외>
죽어도 자이언츠 <할인되지 않는 영화 제외>
에브리띵 윌 체인지 <할인되지 않는 영화 제외>
로드 어헤드 <할인되지 않는 영화 제외>
아르키메데스의 대전 <할인되지 않는 영화 제외>
제목:신 에반게리온 극장판
할인 전 금액 :₩10,000
할인 후 금액 :₩7,500
------------------------------------------------------------------------------------------------------------------------
모자산책 <할인되지 않는 영화 제외>
탑건 매버릭 <할인되지 않는 영화 제외>
공조2: 인터내셔날 <할인되지 않는 영화 제외>
놉 <할인되지 않는 영화 제외>
미니언즈 2 <할인되지 않는 영화 제외>
완벽한 섹스의 맛-감독판 <할인되지 않는 영화 제외>
탑 건 <할인되지 않는 영화 제외>
쥬라기 월드 :도미니언 <할인되지 않는 영화 제외>
범죄도시 2 <할인되지 않는 영화 제외>
아바타 <할인되지 않는 영화 제외>
불릿 트레인  Bullet Train <할인되지 않는 영화 제외>
더 배트맨 <할인되지 않는 영화 제외>
해리포터와 마법사의 돌 <할인되지 않는 영화 제외>
스파이더맨: 노 웨이 홈 Spider-Man: No Way Home <할인되지 않는 영화 제외>
스파이더맨: 노 웨이 홈 Spider

In [81]:
movie.find("span", attrs={"class":"VfPpfd VixbEe"}).get_text()

'₩800'

In [77]:
import pandas as pd

name = []
sale = []
buy = []

for movie in movies :
    title = movie.find(["div","span"], attrs = {"class" :["Epkrse","DdYX5"]}).get_text() 
    
    #할인 전 가격
    ori_pr = movie.find("span", attrs = {"class":"SUZt4c P8AFK"})
    if ori_pr :
        ori_pr = ori_pr.get_text()
    else:
        print(title, "<할인되지 않는 영화 제외>")
        continue
    
    #인기영화는 할인이 없어서 따로 원가 넣겠음(?)
    pop_pr = movie.find("span", attrs = {"class":"w2kbF ePXqnb"})
    if pop_pr : 
        print(title, "<할인되지 않는 영화 제외>")
    
    # 할인된 가격 
    pr = movie.find("span", attrs={"class":"VfPpfd VixbEe"}).get_text()
    
    name.append(title)
    buy.append(ori_pr)
    sale.append(pop_pr)
    
    time.sleep(1)
    
    df = pd.DataFrame(
        {"name":name,
         "buy":buy,
         "sale":sale}
    )

범죄도시 <할인되지 않는 영화 제외>
공조2: 인터내셔날 <할인되지 않는 영화 제외>
레드 아이 <할인되지 않는 영화 제외>
인생은 아름다워 <할인되지 않는 영화 제외>
맞짱 <할인되지 않는 영화 제외>
인생은 뷰티풀: 비타돌체 <할인되지 않는 영화 제외>
블로우백 <할인되지 않는 영화 제외>
투란도트: 어둠의 왕국 <할인되지 않는 영화 제외>
몬스터 피라냐 <할인되지 않는 영화 제외>
돈텔파파 <할인되지 않는 영화 제외>
고속도로 가족 <할인되지 않는 영화 제외>
비스트 <할인되지 않는 영화 제외>
경성별희 <할인되지 않는 영화 제외>
깐부(코로나 방학소동) <할인되지 않는 영화 제외>
죽어도 자이언츠 <할인되지 않는 영화 제외>
에브리띵 윌 체인지 <할인되지 않는 영화 제외>
로드 어헤드 <할인되지 않는 영화 제외>
아르키메데스의 대전 <할인되지 않는 영화 제외>
모자산책 <할인되지 않는 영화 제외>
탑건 매버릭 <할인되지 않는 영화 제외>
공조2: 인터내셔날 <할인되지 않는 영화 제외>
놉 <할인되지 않는 영화 제외>
미니언즈 2 <할인되지 않는 영화 제외>
완벽한 섹스의 맛-감독판 <할인되지 않는 영화 제외>
탑 건 <할인되지 않는 영화 제외>
쥬라기 월드 :도미니언 <할인되지 않는 영화 제외>
범죄도시 2 <할인되지 않는 영화 제외>
아바타 <할인되지 않는 영화 제외>
불릿 트레인  Bullet Train <할인되지 않는 영화 제외>
더 배트맨 <할인되지 않는 영화 제외>
해리포터와 마법사의 돌 <할인되지 않는 영화 제외>
스파이더맨: 노 웨이 홈 Spider-Man: No Way Home <할인되지 않는 영화 제외>
스파이더맨: 노 웨이 홈 Spider-Man: No Way Home <할인되지 않는 영화 제외>
토르: 러브 앤 썬더 <할인되지 않는 영화 제외>
수퍼 소닉 2 [Sonic the Hedgehog 2] <할인되지 않는 영화 제외>
블랙 팬서 (자막판) <할인되지 않는 영화 제외>
듄 <할인되지 않는 영화 제외>
어벤져스 : 엔드게임 (자

In [78]:
df

,name,buy,sale
0,신 에반게리온 극장판,"₩10,000",None
1,트리플엑스 리턴즈,"₩5,000",None
2,클루리스,"₩5,000",None
3,콜래트럴,"₩5,000",None
4,터미널,"₩5,000",None
...,...,...,...
59,이퀼리브리엄,"₩1,400",None
60,잭 리처: 네버 고 백,"₩1,400",None
61,미션임파서블 : 고스트 프로토콜,"₩1,400",None
62,친구와 연인사이,"₩1,400",None


In [ ]:
a = []
